In [1]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
import cv2
import numpy as np
import torch
from collections import deque
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model = YOLO('yolov8n-pose.pt')
swing_detect_model = load_model('/content/drive/MyDrive/swing_detect.h5')
shot_class_model =load_model('/content/drive/MyDrive/swing_class.h5')

100%|██████████| 6.52M/6.52M [00:00<00:00, 22.1MB/s]


In [5]:
swing_detect_class=['swing_begin','swing_middle','swing_end']
shot_class = ['forehand_stroke', 'forehand_slice', 'forehand_volley', 'backhand_stroke', 'backhand_volley', 'backhand_slice']

In [6]:

"""def preprocess_frame(frame):
    frame = cv2.resize(frame, (640, 640))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = frame.astype(np.float32) / 255.0
    frame = np.expand_dims(frame, axis=0)
    frame = np.transpose(frame, (0, 3, 1, 2))
    return frame"""

"""def extract_keypoints(frames):
    all_keypoints = []
    device = torch.device('cuda')
    if not isinstance(frames, list):
        frames = [frames]
    for frame in frames:
        frame = preprocess_frame(frame)
        frame = torch.tensor(frame, dtype=torch.float32).to(device)
        results = model(frame, verbose=False)

        if len(results) > 0 and len(results[0].keypoints) > 0:
            keypoints_list = results[0].keypoints
            bboxes = results[0].boxes
            if bboxes is not None and len(bboxes) > 0:
                try:
                    xyxy = bboxes.xyxy.cpu().numpy()
                    areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in xyxy]
                    max_area_index = np.argmax(areas)
                    keypoints = keypoints_list[max_area_index].xy[0].cpu().numpy()
                    all_keypoints.append(keypoints)
                except IndexError as e:
                    print(f"Error calculating areas: {e}")
                    print(f"Bounding boxes: {bboxes}")
                    continue
    return all_keypoints, frames"""

"""def normalize_keypoints(keypoints):
    hip_index = 11  # index of left hip
    shoulder_index = 5  # index of shoulder

    normalized_keypoints = []
    for frame_keypoints in keypoints:
        if len(frame_keypoints) > max(hip_index, shoulder_index):
            hip_point = frame_keypoints[hip_index]
            shoulder_point = frame_keypoints[shoulder_index]
            relative_points = frame_keypoints - hip_point
            scale_factor = np.linalg.norm(shoulder_point - hip_point)
            if scale_factor != 0:
                relative_points /= scale_factor
            normalized_keypoints.append(relative_points)
        else:
            print(f"Warning: Frame with insufficient keypoints detected. Skipping this frame.")

    return np.array(normalized_keypoints)"""

'def normalize_keypoints(keypoints):\n    hip_index = 11  # index of left hip\n    shoulder_index = 5  # index of shoulder\n\n    normalized_keypoints = []\n    for frame_keypoints in keypoints:\n        if len(frame_keypoints) > max(hip_index, shoulder_index):\n            hip_point = frame_keypoints[hip_index]\n            shoulder_point = frame_keypoints[shoulder_index]\n            relative_points = frame_keypoints - hip_point\n            scale_factor = np.linalg.norm(shoulder_point - hip_point)\n            if scale_factor != 0:\n                relative_points /= scale_factor\n            normalized_keypoints.append(relative_points)\n        else:\n            print(f"Warning: Frame with insufficient keypoints detected. Skipping this frame.")\n\n    return np.array(normalized_keypoints)'

In [7]:
"""def rescale_keypoints(keypoints, input_size, output_size):
  scale_x = output_size[0] / input_size[0]
  scale_y = output_size[1] / input_size[1]
  rescaled_keypoints = []
  for kp in keypoints:
    rescaled_keypoints.append([kp[0] * scale_x, kp[1] * scale_y])
  return rescaled_keypoints"""

"""def draw_keypoints(frame, original_keypoints, color=(0, 255, 0), radius=5):
	for kp in original_keypoints:
		if kp is not None and len(kp) == 2:
			x, y = int(kp[0]), int(kp[1])
			cv2.circle(frame, (x, y), radius, color, -1)
	return frame

def draw_rec(all_keypoints,frames):
	annotated_frames = []
	for point in all_keypoints:
		x, y = point[:2]
		cv2.circle(annotated_frame, (int(x), int(y)), 5, (0, 255, 0), -1)
		annotated_frames.append(annotated_frame)
	return"""

def add_text (new_frame):
  texted_frame = cv2.putText(new_frame, "Swing", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  return texted_frame






In [8]:
"""class DataGenerator(Sequence):
	def __init__(self, x_set, batch_size):
		self.x = x_set
		self.batch_size = batch_size

	def __len__(self):
		return int(np.ceil(len(self.x) / float(self.batch_size)))

	def __getitem__(self, idx):
		batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
		return batch_x"""

'class DataGenerator(Sequence):\n\tdef __init__(self, x_set, batch_size):\n\t\tself.x = x_set\n\t\tself.batch_size = batch_size\n\n\tdef __len__(self):\n\t\treturn int(np.ceil(len(self.x) / float(self.batch_size)))\n\n\tdef __getitem__(self, idx):\n\t\tbatch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]\n\t\treturn batch_x'

In [23]:
video_path = "/content/drive/MyDrive/test_video.mp4"

In [24]:
from google.colab.patches import cv2_imshow

In [25]:
def extract_keypoints(results):
    all_keypoints = []
    if len(results) > 0 and len(results[0].keypoints) > 0:
        keypoints_list = results[0].keypoints
        bboxes = results[0].boxes  # バウンディングボックスのリスト

        if bboxes is not None and len(bboxes) > 0:
            # バウンディングボックスの面積を計算
            try:
                xyxy = bboxes.xyxy.cpu().numpy()  # バウンディングボックスの座標を取得
                areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in xyxy]
            except IndexError as e:
                print(f"Error calculating areas: {e}")
                print(f"Bounding boxes: {bboxes}")
                return all_keypoints

            # 最も大きいバウンディングボックスのインデックスを取得
            max_area_index = np.argmax(areas)

            # 最も大きいバウンディングボックスに対応するキーポイントを取得
            keypoints = keypoints_list[max_area_index].xy[0].cpu().numpy()

            all_keypoints.append(keypoints)
        else:
            print(f"Warning: No bounding boxes found in frame.")
    else:
        print(f"Warning: No keypoints found in frame.")
    return all_keypoints

"""def process_video(input_path, output_folder):
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    all_keypoints = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        frame_keypoints = extract_keypoints(results)
        all_keypoints.extend(frame_keypoints)

        plotted_frame = results[0].plot()
        out.write(plotted_frame)
        frame_count += 1

        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames, detected keypoints in {len(all_keypoints)} frames")

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video processing complete. Output saved to {output_path}")
    print(f"Processed {frame_count} frames, detected keypoints in {len(all_keypoints)} frames")
    return output_path, all_keypoints"""

def process_frame(frame_buffer):
    original_keypoints = []
    plotted_frames = []  # リストとして初期化

    for frame in frame_buffer:
        results = model(frame)
        frame_keypoints = extract_keypoints(results)
        original_keypoints.extend(frame_keypoints)

        plotted_frame = results[0].plot()
        plotted_frames.append(plotted_frame)  # extend ではなく append を使用
    return original_keypoints, plotted_frames


def normalize_keypoints(keypoints):
    hip_index = 11
    shoulder_index = 5

    normalized_keypoints = []
    #print("keypoints is passed")
    for frame_keypoints in keypoints:
        if len(frame_keypoints) > max(hip_index, shoulder_index):
            hip_point = frame_keypoints[hip_index][:2]  # x, y座標のみを使用
            #print(hip_point)
            shoulder_point = frame_keypoints[shoulder_index][:2]  # x, y座標のみを使用
            #print(shoulder_point)

            relative_points = frame_keypoints[:, :2] - hip_point
            #print("relative_points")
            #print(relative_points)

            scale_factor = np.linalg.norm(shoulder_point - hip_point)
            if scale_factor != 0:
                relative_points /= scale_factor

            normalized_keypoints.append(relative_points)
        else:
            print(f"Warning: Frame with insufficient keypoints detected. Keypoints: {len(frame_keypoints)}")

    print(f"Normalized {len(normalized_keypoints)} frames out of {len(keypoints)}")
    return np.array(normalized_keypoints)


def add_text (new_frame):
  texted_frame = cv2.putText(new_frame, "Swing", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  return texted_frame

def add_shot (new_frame,shot_prediction):
  if (shot_prediction == 0):
    texted_frame = cv2.putText(new_frame, "Forehand-stroke", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  elif (shot_prediction == 1):
    texted_frame = cv2.putText(new_frame, "Forehand-slice", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  elif (shot_prediction == 2):
    texted_frame = cv2.putText(new_frame, "Forehand-volley", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  elif (shot_prediction == 3):
    texted_frame = cv2.putText(new_frame, "Backhand-stroke", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  elif (shot_prediction == 4):
    texted_frame = cv2.putText(new_frame, "Backhand-volley", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  else:
    texted_frame = cv2.putText(new_frame, "Backhand-slice", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  return texted_frame


def pad_sequences(sequences, max_length):
    return [seq[:max_length] if len(seq) > max_length else np.pad(seq, ((0, max_length - len(seq)), (0, 0), (0, 0)), 'constant') for seq in sequences]


In [26]:
cap = cv2.VideoCapture(video_path)
frame_buffer = deque(maxlen=40)
is_recording_swing = False
swing_record = []
swing_frame = []
flag = False
final_frames = []

# VideoWriterの設定
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
#width, height = 640, 640
original_width, original_height = 1280, 720
#print(f"Frame size: {original_width}x{original_height}")
out = cv2.VideoWriter('test_outcome.mp4', fourcc, fps, (original_width, original_height))

if not out.isOpened():
    print("Error: VideoWriter not opened.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_buffer.append(frame)

    if (len(frame_buffer) < 40 & flag == False):
        new_frame = frame_buffer[-1]
        original_keypoints, plotted_frame = process_frame([new_frame])
        #print("--------------------------------------------------------------------------------------------------------------------------------------")
        print("keypoints num")
        print(len(original_keypoints))
        #original_keypoints, plotted_frame = process_frame([new_frame])
        if original_keypoints:
            final_frames.append(plotted_frame)

    elif len(frame_buffer) == 40:
        flag = True
        original_keypoints, plotted_frames = process_frame(list(frame_buffer))
        #print("--------------------------------------------------------------------------------------------------------------------------------------")
        print("keypoints num")
        print(len(original_keypoints[0]))
        if len(original_keypoints) > 0:
            normalized_keypoints = normalize_keypoints(original_keypoints)   #ここらへんswing_classで使う  どの形で増やしていってswing終了判定時にX_arrayに持っていくか
            X_array = np.array(normalized_keypoints)
            X_array = X_array.reshape((1, X_array.shape[0], X_array.shape[1] * X_array.shape[2]))

            predictions = swing_detect_model.predict(X_array)
            swing_prediction = np.argmax(predictions, axis=1)

            if swing_prediction[0] != 2:   #スイング中
                print("swinging")
                is_recording_swing = True
                if (len(swing_frame)==0):
                  #print(final_frames)
                  #final_frames = final_frames[:-38]
                  swing_frame.extend(list(plotted_frames)[-2:])
                  swing_record.extend(list(normalized_keypoints)[-2:])

                  #swing_frame.extend(list(plotted_frames)[-2:])  たぶんこれ間違い??
                else:
                  swing_frame.extend(list(plotted_frames)[-2:])
                  swing_record.extend(list(normalized_keypoints)[-2:])

            elif swing_prediction[0] == 2:  # スイング中でない
                if is_recording_swing:  #スイング終了
                    swing_frame.extend(list(plotted_frames)[-2:])
                    swing_record.extend(list(normalized_keypoints)[-2:])
                    is_recording_swing = False
                    normalized_keypoints = normalize_keypoints(swing_record)   #ここらへんswing_classで使う  どの形で増やしていってswing終了判定時にX_arrayに持っていくか
                    X_padded = pad_sequences([normalized_keypoints], 100)
                    X_array = np.array(X_padded).reshape((1, 100, 34))
                    print("--------------------------------------------------------------------------------------------------------------------------------------")
                    print(X_array.shape)
                    predictions = shot_class_model.predict(X_array)
                    shot_prediction = np.argmax(predictions, axis=1)
                    for new_frame in swing_frame:
                        new_frame = add_shot(new_frame,shot_prediction[0])
                        final_frames.append(new_frame)
                    swing_frame.clear()
                    swing_record.clear()

                else: #停止中
                    end_swing = []
                    is_recording_swing = False
                    end_swing.extend(list(plotted_frames)[-2:])
                    #end_swing = [cv2.resize(frame, (original_width, original_height)) for frame in list(frames)[:]]   試してみる価値あり
                    for new_frame in end_swing:
                        final_frames.append(new_frame)

        for _ in range(2):
            if frame_buffer:
                frame_buffer.popleft()
for gen_frame in final_frames:
  out.write(gen_frame)

# リソースの解放
cap.release()
out.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.

0: 384x640 1 person, 10.5ms
Speed: 2.0ms preprocess, 10.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.4ms
Speed: 1.9ms preprocess, 10.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 12.1ms
Speed: 1.9ms preprocess, 12.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
keypoints num
17
Normalized 40 frames out of 40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

0: 384x640 1 person, 18.8ms
Speed: 2.4ms preprocess, 18.8ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.9ms
Speed: 2.0ms preprocess, 11.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 17.2ms
Speed: 2.4ms preprocess, 17.2ms inference, 2.8ms postprocess per image at shap

In [28]:
!cp /content/test_outcome.mp4 /content/drive/MyDrive/

In [ ]:
import cv2
import numpy as np
import torch
from collections import deque



cap = cv2.VideoCapture(video_path)
frame_buffer = deque(maxlen=40)
is_recording_swing = False
swing_record = []
flag = False

# VideoWriterの設定
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
width, height = 640, 640
original_width, original_height = 1280, 720
print(f"Frame size: {original_width}x{original_height}")
out = cv2.VideoWriter('output.mp4', fourcc, fps, (original_width, original_height))

if not out.isOpened():
    print("Error: VideoWriter not opened.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_buffer.append(frame)

    if (len(frame_buffer) < 40 & flag == False):
        new_frame = frame_buffer[-1]
        keypoints, _ = extract_keypoints([new_frame])
        if keypoints:
            original_keypoints = rescale_keypoints(keypoints[0], (640, 640), (original_width, original_height))
            new_frame = draw_keypoints(new_frame, original_keypoints, color=(0, 255, 0), radius=5)
            out.write(new_frame)

    elif len(frame_buffer) == 40:
        flag = True
        keypoints, frames = extract_keypoints(list(frame_buffer))
        if len(keypoints) > 0:
            normalized_keypoints = normalize_keypoints(keypoints)
            X_array = np.array(normalized_keypoints)
            X_array = X_array.reshape((1, X_array.shape[0], X_array.shape[1] * X_array.shape[2]))

            # DataGenerator を使用せず、直接 X_array を predict() に渡す
            predictions = swing_detect_model.predict(X_array)
            swing_prediction = np.argmax(predictions, axis=1)

            if swing_prediction[0] != 2:   #スイング中
                is_recording_swing = True
                print("swinging")
                swing_record.extend([cv2.resize(frame, (original_width, original_height)) for frame in list(frames)[-2:]])

            elif swing_prediction[0] == 2:  # スイング中でない
                if is_recording_swing:  #スイング終了
                    swing_record.extend([cv2.resize(frame, (original_width, original_height)) for frame in list(frames)[-2:]])
                    is_recording_swing = False
                    for new_frame, kp in zip(swing_record, keypoints[-len(swing_record):]):
                        original_keypoints = rescale_keypoints(kp, (640, 640), (original_width, original_height))
                        #ここで推論モデルを挟むだけ(padする)
                        new_frame = draw_keypoints(new_frame, original_keypoints, color=(0, 255, 0), radius=5)
                        new_frame = add_text(new_frame)
                        out.write(new_frame)
                    swing_record.clear()

                else: #停止中
                    is_recording_swing = False
                    end_swing = [cv2.resize(frame, (original_width, original_height)) for frame in list(frames)[-2:]]
                    #end_swing = [cv2.resize(frame, (original_width, original_height)) for frame in list(frames)[:]]   試してみる価値あり
                    for new_frame, kp in zip(end_swing, keypoints[-2:]):
                        original_keypoints = rescale_keypoints(kp, (640, 640), (original_width, original_height))
                        new_frame = draw_keypoints(new_frame, original_keypoints, color=(0, 255, 0), radius=5)
                        out.write(new_frame)

        for _ in range(2):
            if frame_buffer:
                frame_buffer.popleft()

# リソースの解放
cap.release()
out.release()
cv2.destroyAllWindows()

29.97002997002997
Frame size: 1280x720
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
swinging
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
swinging
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
swinging
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
swinging
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━